## Principal Component Analysis (PCA)
---

### Learning Objectives

- Describe what PCA does and what it is used for in data science.
- Practice computing PCA with sklearn.
- Interpret PCA results graphically.

### Lesson Guide
- [Motivation](#motivation)
- [What is PCA?](#whatispca)
    - [Eigenvalues and Eigenvectors](#eigenpairs)
    - [Principal Components](#pcs)
- [Manual PCA Codealong](#manual-codealong)
    - [1. Basic EDA](#basic-eda)
    - [2. Subset and Normalize](#subset)
    - [3. Find the Correlation Matrix](#corr)
    - [4. Eignenvalues and Eigenvectors](#eigen)
    - [5. Explained Variance](#var)
    - [6. Projection Matrix W](#projection)
    - [7. Transformed Matrix Z](#transformed)
- [More Reading](#more-reading)

# PCA = Principle Component Analysis

**Goals**: 

- *Transform* original variable/features into new, "high-performance" features
- *Reduce* the dimensionality of the data
- *Eliminate* multicollinearity


<a id="motivation"></a>
## Motivation

### Dimensionality Reduction

Dimensionality reduction reduces the number of random variables that you are considering for analysis until you are left with the most important variables.

> Dimensionality reduction is not an end goal in itself, but a tool to form a dataset with more parsimonious features for further visualization and/or modelling.

### Reducing Colinearity in Input

To get a quick summary of our data, we can calculate a covariance matrix, an unstandardized correlation matrix.

The diagonal elements in a covariance matrix show us the variance of each of our features.

The off-diagnal elements show the covariance, the amount of colinearity and redundancy between our variables.

## Motivating Example

Say that I want to predict **age** from *stress*, *income* and *health*.

1. Three-dimensional data
2. Multicollinearity probably exists

*PCA* will give me one or two **super-predictor** variables called *components* (hopefully).


<a id="whatispca"></a>
## What is PCA?

---

PCA is the quintessential "dimensionality reduction" algorithm. 

_Dimensionality reduction_ is the process of combining or collapsing the existing features (columns in X) into fewer features. 

These hopefully:

- Retain the signal in the original data, and
- Reduce noise.

---

**Essentially...**

- PCA finds *linear combinations* of current predictor variables that...
- create new "principal components". The principal components explain...
- the maximum possible amount of variance in your predictors.

$$ PC1 = w_{1,1}(\text{stress}) + w_{2,1}(\text{income}) + w_{3,1}(\text{health})$$

$$ PC2 = w_{1,2}(\text{stress}) + w_{2,2}(\text{income}) + w_{3,2}(\text{health})$$

$$ PC3 = w_{1,3}(\text{stress}) + w_{2,3}(\text{income}) + w_{3,3}(\text{health})$$

This is cool because...

- $PC1$ is more important than $PC2$ is better than $PC3$  
- All of these  $PCs$ are *uncorrelated*

---

**Visually...**

> Think of PCA as a coordinate transformation.  The old axes are the original variables (columns). The new axes are the principal components from PCA.

**The new axes (principal components) become the  most concise, informative descriptors of our data as a whole.**


<img alt="orthogonal eigenvectors" src="https://drive.google.com/uc?export=view&id=117wER1h6aWurSSjghUUNjPBqpm9xydl4">


<img alt="[transformed XY" src="https://drive.google.com/uc?export=view&id=1UfoiTOatAFGybhf3Kg8UG21s8ijw9ufY">

<a id="pcs"></a>
### Principal Components

---

- We are looking for new *directions* in feature space
- Each consecutive direction tries to maximize *remaining variance*
- Each direction is *orthogonal* to all the others

**These new *directions* are the "principal components", i.e. the new coordinate system for your data.**

> Applying PCA to your data *transforms* your original data columns (variables) onto the new principal component axes.


(Did you catch that?  The *variables* define the *cordinate system*.)

(CP1) **Let's review...**

**The PCA transformation creates new variables that...**
1. Optimize "explained variance", and
2. Are uncorrelated.

Creating these variables is a well-defined mathematical process. In essence, **each component is created as a weighted sum of your original columns, such that all components are orthogonal (perpendicular) to each other**.

#### Example Continued

The inputs `stress`, `income` and `health`...

...can be *replaced* with 3 *new* variables...

- `PC1` $\rightarrow$ most variance
- `PC2`
- `PC3` $\rightarrow$ least variance (noise?)

---

Mathematically:

$$ PC1 = w_{1,1}(\text{stress}) + w_{2,1}(\text{income}) + w_{3,1}(\text{health})$$

$$ PC2 = w_{1,2}(\text{stress}) + w_{2,2}(\text{income}) + w_{3,2}(\text{health})$$

$$ PC3 = w_{1,3}(\text{stress}) + w_{2,3}(\text{income}) + w_{3,3}(\text{health})$$

The weights are called *loadings*... they are coefficients indicating how heavily each of the input data are weighted

e.g.

$$ PC1 = 0.01(\text{stress}) - 0.54(\text{income}) + 0.71(\text{health})$$

> We will see how to get these values from `sklearn`


### Capturing variance

The total variance of your data gets redistributed among the principal components:

$$\text{var}(PC1) > \text{var}(PC2) > \text{var}(PC3)$$


> #### Interpreting PCA: Signal v. Noise

> PCA attempts to *maximize signal* (high variance) while *isolating noise* (low variance)

> - Most variance captured in first several principal components
> - Noise isolated to last several principal compoments
> - This done simultaneously across *all input variables*


### Isolating variance

There is no covariance between principal components

$$\text{covar}(PC1, PC2) = 0$$

$$\text{covar}(PC1, PC3) = 0$$

$$\text{covar}(PC2, PC3) = 0$$


**Two assumptions that PCA makes:**
1. **Linearity:** The data does not hold nonlinear relationships.
2. **Large variances define importance:** The dimensions are constructed to maximize remaining variance.

**PRINCIPAL COMPONENT TRANSFORMATION OF DATA: PC1 VS PC2**

[setosa.io has an extremely nice interactive visualization for PCA](http://setosa.io/ev/principal-component-analysis/)

---

<img alt="pca_coordinate_transformation" src="https://drive.google.com/uc?export=view&id=1EoJBgCaeghbJGbx7SVwtx0IIUcji07lv">

### How we derive PCA
PCA relies on the...

**Eigenvalue decomposition of the covariance matrix**
This diagonalizes the covariance matrix, therby eliminating covariance.

**The principal component transformation**
This transforms each input variable $X$ onto a new orthogonal basis in which the new variables $Z$ are maximally variant.

$$ \mathbf{Z = WX} $$

> * Sigma = Covariance Matrix (X)// X is the original feature data
> * [U,S,V] = svd(Sigma)//Perform Singular Value Decomposition
> * Ureduce = U(:,1:k) // k is the number of PCA components
> * W = Ureduce<sup>T</sup>


### Why would we want to do PCA?

---

- We can **reduce the number of dimensions** (remove less important components), while losing mostly noise rather than signal.
- Since we are assuming our variables are interrelated (at least in the sense that they together explain a dependent variable), the information of interest should exist along directions with largest variance.
- The directions of largest variance should have the highest signal-to-noise ratio.
- Correlated predictor variables (also referred to as "redundancy" of information) are combined into independent variables. Our predictors from PCA are guaranteed to be independent.

---

[Good paper on PCA](http://arxiv.org/pdf/1404.1100.pdf)

[Nice site on how PCA is done step by step with coding](http://sebastianraschka.com/Articles/2015_pca_in_3_steps.html#pca-vs-lda)

<a id="manual-codealong"></a>
## PCA Codealong

---

**DATA**

We are going to be using a simple 75-row, 4-column dataset with demographic information. It contains:

    age (limited to 20-65)
    income
    health (a rating on a scale of 1-100, where 100 is the best health)
    stress (a rating on a scale of 1-100, where 100 is the most stressed)
    
All of the variables are continuous.

---

In [1]:
#1 Code to read csv file into colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

     |████████████████████████████████| 993kB 2.8MB/s 


In [0]:
#2. Get the file
#make sure you upload all your data files to your Google drive and change share->Advanced->change->anyone with the link can view
downloaded = drive.CreateFile({'id':'1FRTKbjLnzxDp0mJ3WHaFO-hYmwfXCW-V'}) # replace the id with id of file you want to access
downloaded.GetContentFile('simple_demographics.csv')

In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

sns.set_style('white')

demo = pd.read_csv('simple_demographics.csv')
demo.shape

(75, 4)

In [19]:
demo.head()

,age,health,income,stress
0,21,74.0,42746.0,53.0
1,33,64.0,72792.0,49.0
2,30,78.0,74178.0,64.0
3,32,71.0,102548.0,63.0
4,57,46.0,120418.0,35.0


<a id="basic-eda"></a>
### 1. Basic EDA

Make a seaborn pairplot for the dataset to see how each feaure is correlated to the other.

---

<a id="subset"></a>
### 2. Subset and normalize

Subset the data to only include:

    income
    health
    stress

We will be comparing the principal components to age specifically, so we are leaving age out.

---

In [0]:
demo_noage = demo[['health','income','stress']]
#Let's normalize the data
demo_noage = (demo_noage - demo_noage.mean()) / demo_noage.std()

<a id="corr"></a>
### 3. Calculate correlation matrix

We will be using the correlation matrix to calculate the eigenvectors and eigenvalues.

---

In [5]:
demo_noage_corr = np.corrcoef(demo_noage.values.T)
demo_noage.corr()

,health,income,stress
health,1.000000,0.192037,0.527663
income,0.192037,1.000000,-0.347925
stress,0.527663,-0.347925,1.000000


<a id="eigen"></a>
### 4. Calculate the principal components and associated explained variances

---

In [6]:
from sklearn.decomposition import PCA

X = demo_noage

pca = PCA()
pca = pca.fit(X)

print(pca.explained_variance_)
print(pca.components_)

[1.55645677 1.17357375 0.26996948]
[[-0.6187659   0.25173885 -0.74414804]
 [ 0.5126449   0.84716255 -0.13968116]
 [-0.59525118  0.46791364  0.65324793]]


Interpreting the "components": recall that

$$ PC1 = w_{1,1}(\text{health}) + w_{2,1}(\text{income}) + w_{3,1}(\text{stress})$$

$$ PC2 = w_{1,2}(\text{health}) + w_{2,2}(\text{income}) + w_{3,2}(\text{stress})$$

$$ PC3 = w_{1,3}(\text{health}) + w_{2,3}(\text{income}) + w_{3,3}(\text{stress})$$


<a id="transformed"></a>
### 5. Construct the Transformed Data Set $Z$

---

In [7]:
Z = pca.transform(demo_noage)

features_pca = ['PC'+str(i+1) for i in range(pca.n_components_)]
Z = pd.DataFrame(Z, columns=features_pca)
Z.head(5)

,PC1,PC2,PC3
0,-0.882350,-0.150070,-1.166109
1,-0.104542,0.102205,-0.597921
2,-1.372329,0.496715,-0.563529
3,-0.867671,0.797774,-0.017223
4,1.566225,0.491018,0.087187


In [8]:
Z.shape

(75, 3)

In [9]:
Z.corr()

,PC1,PC2,PC3
PC1,1.000000e+00,7.531902e-16,6.017637e-17
PC2,7.531902e-16,1.000000e+00,1.239421e-16
PC3,6.017637e-17,1.239421e-16,1.000000e+00


### 6. PCA applied to prediction problems

Now build out a base-line linear regression model predicting `age` from `health`, `income` and `stress`.

In [20]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

features = ['health', 'income', 'stress']

X = demo[features]
y = demo['age']

ss = StandardScaler()
Xs = ss.fit_transform(X)

lr = LinearRegression()

print('Cross validataion score with Linear Regression: %.2f%%' %(cross_val_score(lr, Xs, y, cv=5).mean()*100))

Cross validataion score with Linear Regression: 85.56%


### Repeat the linear regression, but reduce the dimensionality to 2 (instead of 3) using PCA.  

In [11]:
from sklearn.decomposition import PCA

X = demo[features]
y = demo['age']

ss = StandardScaler()
Xs = ss.fit_transform(X)

pca = PCA(n_components=2)
Xt = pca.fit_transform(Xs)

lr = LinearRegression()

print('Cross validataion score with Linear Regression and 2 PCA components: %.2f%%' %(cross_val_score(lr, Xt, y, cv=5).mean()*100))

pd.DataFrame(pca.components_.T, columns=['PC1', 'PC2'], index=features)

Cross validataion score with Linear Regression and 2 PCA components: 72.24%


,PC1,PC2
health,-0.618766,0.512645
income,0.251739,0.847163
stress,-0.744148,-0.139681


In [0]:
def pca_performance(Xs,n):
  
#write a function to try out different PCA components
#print out the prediction
  
  pca = PCA(n_components=n)
  Xt = pca.fit_transform(Xs)

  lr = LinearRegression()

  print('Cross validataion score with Linear Regression and %d PCA components: %.2f%%' %(n,cross_val_score(lr, Xt, y, cv=5).mean()*100))

#   pd.DataFrame(pca.components_.T, columns=['PC1', 'PC2'], index=features)

In [33]:
pca_performance(Xs,1)
pca_performance(Xs,2)
pca_performance(Xs,3)

Cross validataion score with Linear Regression and 1 PCA components: 69.92%
Cross validataion score with Linear Regression and 2 PCA components: 72.24%
Cross validataion score with Linear Regression and 3 PCA components: 85.56%


## Apply PCA to Digits Dataset

In [0]:
# starter code 
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
digits = load_digits()

In [0]:
X_train, X_test, y_train, y_test = train_test_split(digits.data, digits.target, test_size=0.2)

In [0]:
# normalizing the dataset 
ss = StandardScaler()
X_train = ss.fit_transform(X_train)
X_test = ss.transform(X_test)

In [16]:
%%time
#Let's see how good is Logistic Regression
logit = LogisticRegression()
logit.fit(X_train, y_train)
print ("Logistic Regression Accuracy: %.2f%%" %(logit.score(X_test, y_test)*100))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Logistic Regression Accuracy: 98.06%
CPU times: user 410 ms, sys: 1.97 ms, total: 412 ms
Wall time: 419 ms


In [17]:
%%time
from sklearn.decomposition import PCA
pca = PCA(n_components=16)
X_train_t = pca.fit_transform(X_train)
X_test_t = pca.transform(X_test)
logit = LogisticRegression()
logit.fit(X_train_t, y_train)
print ("Logistic Regression Accuracy with %d PCA components: %.2f%%" %(16,logit.score(X_test_t, y_test)*100))

Logistic Regression Accuracy with 16 PCA components: 94.17%
CPU times: user 135 ms, sys: 87 ms, total: 222 ms
Wall time: 122 ms


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [0]:
def digit_pca(X_train,X_test,y_train,y_test, n):
  pca = PCA(n_components=n)
  X_train_t = pca.fit_transform(X_train)
  X_test_t = pca.transform(X_test)
  logit = LogisticRegression()
  logit.fit(X_train_t, y_train)
  print ("Logistic Regression Accuracy with %d PCA components: %.2f%%" %(16,logit.score(X_test_t, y_test)*100))

In [36]:
%%time
digit_pca(X_train,X_test,y_train,y_test, 64)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Logistic Regression Accuracy with 16 PCA components: 98.06%
CPU times: user 477 ms, sys: 76.1 ms, total: 553 ms
Wall time: 485 ms


In [37]:
%%time
digit_pca(X_train,X_test,y_train,y_test, 32)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Logistic Regression Accuracy with 16 PCA components: 95.83%
CPU times: user 266 ms, sys: 112 ms, total: 378 ms
Wall time: 263 ms


In [38]:
%%time
digit_pca(X_train,X_test,y_train,y_test, 16)

Logistic Regression Accuracy with 16 PCA components: 94.17%
CPU times: user 151 ms, sys: 83.6 ms, total: 235 ms
Wall time: 128 ms


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [39]:
%%time
digit_pca(X_train,X_test,y_train,y_test, 8)

Logistic Regression Accuracy with 16 PCA components: 88.06%
CPU times: user 90.2 ms, sys: 68.3 ms, total: 158 ms
Wall time: 92.6 ms


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


## Digit recognition with Manifold Learning

<img src="http://benalexkeen.com/wp-content/uploads/2017/05/isomap.png" style="float: left; margin: 20px; height: 75px">

In [0]:
from sklearn.manifold import Isomap
def digit_manifold(X_train,X_test,y_train,y_test, n):
  model = Isomap(n_components=n)
  X_train_t = model.fit_transform(X_train)
  X_test_t = model.transform(X_test)
  logit = LogisticRegression()
  logit.fit(X_train_t, y_train)
  print ("Logistic Regression Accuracy with %d Isomap components: %.2f%%" %(n,logit.score(X_test_t, y_test)*100))

In [41]:
%%time
digit_manifold(X_train,X_test,y_train,y_test, 64)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Logistic Regression Accuracy with 64 Isomap components: 97.78%
CPU times: user 2.75 s, sys: 334 ms, total: 3.08 s
Wall time: 2.61 s


In [42]:
%%time
digit_manifold(X_train,X_test,y_train,y_test, 32)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Logistic Regression Accuracy with 32 Isomap components: 96.11%
CPU times: user 2.58 s, sys: 275 ms, total: 2.85 s
Wall time: 2.24 s


In [43]:
%%time
digit_manifold(X_train,X_test,y_train,y_test, 16)

Logistic Regression Accuracy with 16 Isomap components: 95.28%
CPU times: user 2.32 s, sys: 203 ms, total: 2.52 s
Wall time: 1.99 s


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [44]:
%%time
digit_manifold(X_train,X_test,y_train,y_test, 5)

Logistic Regression Accuracy with 5 Isomap components: 91.11%
CPU times: user 1.59 s, sys: 111 ms, total: 1.7 s
Wall time: 1.54 s


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


<a id="more-reading"></a>
### More useful links, reading, and references for images

---

[PCA 4 dummies](https://georgemdallas.wordpress.com/2013/10/30/principal-component-analysis-4-dummies-eigenvectors-eigenvalues-and-dimension-reduction/)

[Stackoverflow making sense of PCA](http://stats.stackexchange.com/questions/2691/making-sense-of-principal-component-analysis-eigenvectors-eigenvalues)

[PCA and spectral theorem](http://stats.stackexchange.com/questions/217995/what-is-an-intuitive-explanation-for-how-pca-turns-from-a-geometric-problem-wit)

[PCA in 3 steps: eigendecomposition and SVD](http://sebastianraschka.com/Articles/2015_pca_in_3_steps.html#pca-vs-lda)

[Tutorial on PCA](http://arxiv.org/pdf/1404.1100.pdf)

[PCA math and examples](http://www.stat.cmu.edu/~cshalizi/uADA/12/lectures/ch18.pdf)